**QUESTION 3**

---
---
---

In [ ]:
import pickle
import numpy as np
from pathlib import Path

from src.input import Input_uc, Input_ed, Input_ed_prev
from src.output import Output_uc, Output_ed, Output_ed_prev
from src.unit_commitment import solve_uc
from src.economic_dispatch import solve_ed, solve_ed_prev

In [ ]:
num_units=122
num_periods=24
num_buses=197

In [ ]:
path_folder_processed_kpg193 = Path.cwd().resolve() / "data" / "input" / "processed" / "KPG193_ver1_2"
timestamp_2022 = np.load(Path.cwd().resolve() / "data" / "input" / "processed" / "timestamp_2022.npy")
time_start, time_end = np.datetime64("2022-07-02T00"), np.datetime64("2022-07-02T23")
idx_time_start, idx_time_end = np.where(timestamp_2022 == time_start)[0][0], np.where(timestamp_2022 == time_end)[0][0]

In [ ]:
unit_type = np.load(path_folder_processed_kpg193 / "unit_type.npy")

In [ ]:
solar_p_max = np.load(path_folder_processed_kpg193 / "solar_p_max_2022.npy")[idx_time_start-1:idx_time_end+1].sum(axis=1)
solar_p_min = np.zeros(num_periods + 1)
wind_p = np.load(path_folder_processed_kpg193 / "wind_p_2022.npy")[idx_time_start-1:idx_time_end+1].sum(axis=1)
hydro_p = np.load(path_folder_processed_kpg193 / "hydro_p_2022.npy")[idx_time_start-1:idx_time_end+1].sum(axis=1)

load = np.load(path_folder_processed_kpg193 / "demand_2022.npy")[idx_time_start-1:idx_time_end+1]
system_reserve_up = np.zeros(num_periods + 1)
system_reserve_down = np.zeros(num_periods + 1)

p_min =  np.load(path_folder_processed_kpg193 / "p_min.npy")
p_max =  np.load(path_folder_processed_kpg193 / "p_max.npy")
ramp_up =  np.load(path_folder_processed_kpg193 / "ramp_up.npy")
ramp_down =  np.load(path_folder_processed_kpg193 / "ramp_down.npy")
startup_ramp =  np.load(path_folder_processed_kpg193 / "startup_ramp.npy")
shutdown_ramp =  np.load(path_folder_processed_kpg193 / "shutdown_ramp.npy")
min_up =  np.load(path_folder_processed_kpg193 / "min_up.npy")
min_down =  np.load(path_folder_processed_kpg193 / "min_down.npy")
cost_quad =  np.load(path_folder_processed_kpg193 / "cost_quad.npy")
cost_lin =  np.load(path_folder_processed_kpg193 / "cost_lin.npy")
cost_const =  np.load(path_folder_processed_kpg193 / "cost_const.npy")

In [ ]:
cost_startup_step = pickle.load(open(path_folder_processed_kpg193 / "cost_startup_step.pkl", "rb"))
num_cooling_steps = np.array([len(csc_i) for csc_i in cost_startup_step])

u_prev = [
    np.load(path_folder_processed_kpg193 / "status_2022.npy")
    [idx_time_start-num_cooling_steps.max():idx_time_start]
    [:, idx_unit][-lookup:].tolist()
    for idx_unit, lookup in enumerate(num_cooling_steps)
]

In [ ]:
mustoff_2022 = np.load(path_folder_processed_kpg193 / "mustoff_2022.npy")
mustoff_2022 = mustoff_2022[
    (mustoff_2022[:, 2] >= idx_time_start) & (mustoff_2022[:, 1] <= idx_time_end)
]
mustoff = []
for unit, start, end in mustoff_2022.tolist():
    start_clipped = max(start, idx_time_start)
    end_clipped = min(end, idx_time_end)
    for t in range(start_clipped, end_clipped + 1):
        mustoff.append((unit, t - int(idx_time_start)))

In [ ]:
input_ed_prev=Input_ed_prev(
    # meta
    num_units=num_units,
    num_buses=num_buses,
    voll=3500*1300,
    let_blackout=False,
    curtail_penalty=0,
    let_curtail=False,
    exact_reserve=False,
    # renewable
    solar_p_max=solar_p_max[0],
    solar_p_min=solar_p_min[0],
    wind_p=wind_p[0],
    hydro_p=hydro_p[0],
    # system
    load=load[0],
    system_reserve_up=system_reserve_up[0],
    system_reserve_down=system_reserve_down[0],
    # u_prev
    u_prev=[u[-1] for u in u_prev],
    # operational
    p_min=p_min, p_max=p_max, cost_quad=cost_quad, cost_lin=cost_lin, cost_const=cost_const,
)
output_ed_prev = Output_ed_prev()
solve_ed_prev(input_ed_prev=input_ed_prev, output_ed_prev=output_ed_prev, only_p_prev=False)
p_prev = output_ed_prev.p

In [ ]:
solar_p_max = solar_p_max[1:]
solar_p_min = solar_p_min[1:]
wind_p = wind_p[1:]
hydro_p = hydro_p[1:]
load = load[1:]
system_reserve_up = system_reserve_up[1:]
system_reserve_down = system_reserve_down[1:]

---
---
---

In [ ]:
system_reserve_up = load.sum(axis=1) * 0.05
system_reserve_down = load.sum(axis=1) * 0.05

In [ ]:
input_uc = Input_uc(
    # meta
    num_units=num_units,
    num_periods=num_periods,
    num_buses=num_buses,
    voll=3500*1300,
    let_blackout=False,
    curtail_penalty=0,
    let_curtail=False,
    exact_reserve=False,
    # renewable
    solar_p_max=solar_p_max,
    solar_p_min=solar_p_min,
    wind_p=wind_p,
    hydro_p=hydro_p,
    # system
    load=load,
    system_reserve_up=system_reserve_up,
    system_reserve_down=system_reserve_down,
    # operational constraint
    p_min=p_min,
    p_max=p_max,
    ramp_up=ramp_up,
    ramp_down=ramp_down,
    startup_ramp=startup_ramp,
    shutdown_ramp=shutdown_ramp,
    min_up=min_up,
    min_down=min_down,
    # generation cost function
    cost_quad=cost_quad,
    cost_lin=cost_lin,
    cost_const=cost_const,
    # previous horizon
    p_prev=p_prev,
    u_prev=u_prev,
    # startup cost function
    cost_startup_step=cost_startup_step,
    # mustoff
    mustoff=mustoff,
)

In [ ]:
output_uc = Output_uc()
solve_uc(input_uc=input_uc, output_uc=output_uc)

In [ ]:
output_ed = Output_ed(num_periods=num_periods, num_units=num_units, num_buses=num_buses)

for time_period in range(num_periods):
    input_ed = Input_ed(
        # meta
        time_period=time_period,
        num_units=num_units,
        num_buses=num_buses,
        voll=3500*1300,
        let_blackout=False,
        curtail_penalty=0,
        let_curtail=False,
        exact_reserve=False,
        # renewable
        solar_p_max=solar_p_max,
        solar_p_min=solar_p_min,
        # uc
        input_uc=input_uc,
        output_uc=output_uc,
    )

    solve_ed(input_ed=input_ed, output_ed=output_ed)

output_ed.compute_auxiliary_results()
output_uc.compute_auxiliary_results(output_ed=output_ed)

In [ ]:
output_uc.system_reserve_up, input_uc.system_reserve_up, output_uc.system_reserve_down, input_uc.system_reserve_down # these are all 24 you get what i mean like the output system rserve is kinda bigger or just equal  we got this 


In [ ]:
path_folder_output = "/disk/disk3/gyeongmin/pse_unit_commitment/data/output"
with open(f"{path_folder_output}/1.pkl", "wb") as f:
    pickle.dump([output_uc, output_uc, output_ed], f)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming output_ed.p is 122 x 24, summing along axis 0 to get total active power (MW)
active_power = np.sum(output_ed.p, axis=0)  # Summing across all units for each time period
load = np.sum(input_uc.load, axis=1)  # Summing load across all buses for each time period (MW)
load= load - reg
# Convert to GW (since 1 GW = 1000 MW)
active_power_gw = active_power / 1000
load_gw = load / 1000

# Convert the reserves from list to numpy arrays (if needed)
reserve_up_input = np.array(input_uc.system_reserve_up)
reserve_down_input = np.array(input_uc.system_reserve_down)
reserve_up_output = np.array(output_uc.system_reserve_up)
reserve_down_output = np.array(output_uc.system_reserve_down)

# Plotting
plt.figure(figsize=(12, 8))

# Set width for the bars
bar_width = 0.35

# Plot active power (MW) and load (MW) side by side
plt.bar(np.arange(24) + bar_width / 2, active_power_gw, width=bar_width, color='blue', alpha=0.7, label="Active Power Dispatch (GW)")
plt.bar(np.arange(24) - bar_width / 2, load_gw, width=bar_width, color='gold', label="Hourly Demand (GW)")

# Plot reserve requirements and actual reserves (in MW)
plt.plot(np.arange(24), reserve_up_input / 1000, color='red', linestyle='--', label="Reserve Up Requirement (GW)")
plt.plot(np.arange(24), reserve_down_input / 1000, color='orange', linestyle='--', label="Reserve Down Requirement (GW)")
plt.plot(np.arange(24), reserve_up_output / 1000, color='green', marker='o', label="Reserve Up Actual (GW)")
plt.plot(np.arange(24), reserve_down_output / 1000, color='purple', marker='o', label="Reserve Down Actual (GW)")

# Set axis labels and title
plt.xlabel('Time Period (Hour)')
plt.ylabel('Power (GW)')
plt.title('Active Power, Spinning Reserve, and Load')

# Show the legend
plt.legend()
plt.xlim(-1, 24)
# Rotate the x-axis labels for better readability
plt.xticks(np.arange(24), [f'{h}:00' for h in range(24)], rotation=45)

# Display the plot
plt.tight_layout()
plt.show()
